In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points

os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')
pd.options.display.max_columns = None

# Load Data
all_flowlines_gdf = gpd.read_file('all_flowlines.geojson')
spills_gdf = gpd.read_file('spills.geojson')

/var/folders/p3/gs65vvgs1wd90mnxb3q3hvy80000gp/T/ipykernel_2532/468511908.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Check size
print(all_flowlines_gdf.shape)
print(spills_gdf.shape)

In [2]:
all_flowlines_gdf.head(2)

,Operator,Fluid,Material,Diam_in,Status,Length_ft,SHAPE_Length,LOCATION_ID,FLOWLINEID,STARTLOCATIONID,FLOWLINEACTION,ENTIRELINEREMOVED,ACTIONDESCRIPTION,RECEIVE_DATE,OPERATOR_NUM,COMPANY_NAME,LOCATIONTYPE,ENDLAT,ENDLONG,STARTLAT,STARTLONG,PIPEMATERIAL,BEDDINGMATERIAL,TYPEOFFLUIDTRANS,MAXOPPRESSURE,geometry
0,EVERGREEN NATURAL RESOURCES LLC,Gas,polly,4.0,Active,2277.71,693.972162,NaN,NaN,NaN,None,None,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,"MULTILINESTRING ((524642.670 4117088.796, 5246..."
1,NOBLE ENERGY INC,Multiphase,Carbon Steel,3.0,Abandoned,651.58,198.525215,NaN,NaN,NaN,None,None,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,"MULTILINESTRING ((527997.281 4463899.920, 5281..."


In [3]:
spills_gdf.head(2)

,trkg_num,Operator Name,facility_type,Spill_Desc,Spill Type,Root Cause,Preventative Measure,Root Cause Type,Detailed Root Cause Type,Long,Lat,facility_status,Metallic?,geometry
0,401530532,K P KAUFFMAN COMPANY INC,FLOWLINE,Historical contamination was discovered during...,1,Facility #7 consolidation line failed do to un...,The damaged section of flowline was repaired a...,Unknown,Unknown,-104.914183,40.019361,CL,Unknown,POINT (507323.245 4429909.631)
1,401524345,GREAT WESTERN OPERATING COMPANY LLC,TANK BATTERY,Soil impacts were discovered during removal of...,1,Unknown,None,Unknown,Unknown,-104.467746,39.602613,AC,Unknown,POINT (545695.434 4383787.964)


In [4]:
def find_nearest_line(spill_row, flowlines_gdf):
    """
    For a given spill, find the nearest flowline from the same operator.
    """
    spill_point = spill_row.geometry
    
    matching_flowlines = flowlines_gdf[flowlines_gdf['Operator'] == spill_row['Operator Name']]
    
    min_dist = float('inf')
    nearest_line = None
    
    for _, flowline_row in matching_flowlines.iterrows():
        dist = spill_point.distance(flowline_row.geometry)
        
        if dist < min_dist:
            min_dist = dist
            nearest_line = flowline_row
    
    return nearest_line

In [5]:
spills_gdf['Nearest Flowline'] = spills_gdf.apply(lambda row: find_nearest_line(row, all_flowlines_gdf), axis=1)

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return